<a href="https://colab.research.google.com/github/IgorSouza21/ComputacaoEvolutiva/blob/master/Computa%C3%A7%C3%A3oEvolutiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages Required

In [0]:
import nltk
nltk.download('all')
!pip install info-gain

# Imports

In [0]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
from collections import defaultdict
import numpy as np
import random as rnd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import string
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import accuracy_score
import os

# Pré-Processamento

In [0]:
neg_words = ['not', 'no', 'nor', "n't"]
def tokenize(t):
    stop = stopwords.words('english')
    x = []
    t = word_tokenize(t)
    for a in t:
        a = a.lower()
        if (a not in stop or a in neg_words) and a not in string.punctuation:
            if a == "n't":
                x.append("not")
            if a.isalpha():
                x.append(WordNetLemmatizer().lemmatize(a))

    return x

def filter_by_pos(x):
    if 'NN' in x[1] or 'VB' in x[1] or 'JJ' in x[1] or "RB" in x[1]:
        return True
    else:
        return False

def get_features(sentence):
    features = defaultdict(int)
    tokens = tokenize(sentence)
    pos = nltk.pos_tag(tokens)
    
    new_pos  = []
    for i in range(len(pos)):
        if 'NN' in pos[i][1] or 'VB' in pos[i][1] or 'JJ' in pos[i][1] or \
           "RB" in pos[i][1]:
            if "VB" in pos[i][1] or "JJ" in pos[i][1] or "RB" in pos[i][1]:
                if pos[i-1] in neg_words:
                    new_pos.append(pos[i][0] + "_NOT")
                else:
                    new_pos.append(pos[i][0])
            else:
                new_pos.append(pos[i][0])
                
#     new_pos = list(map(lambda x: x[0], list(filter(filter_by_pos, pos))))

    for i in range(len(new_pos)):
        features[new_pos[i]] += 1
        if i < (len(new_pos) - 1):
            bigram = "%s__%s" % (new_pos[i], new_pos[i+1])
            features[bigram] += 1

    return features

def get_all_features(data):
    dicts = []

    file = open("features.txt", 'w')
    for d in data:
        feat = get_features(d)
        for f in feat.items():
            for i in range(f[1]):
                file.write(f[0] + " ")
        file.write('\n')
        dicts.append(feat)
    file.close()

def get_features_disk():
    features = []
    file = open("features.txt", 'r')
    for f in file:
        feat = defaultdict(int)
        line = f.strip().split()
        for word in line:
            feat[word] += 1
        features.append(feat)
    file.close()
    dataframe = pd.DataFrame(features)
    dataframe = dataframe.fillna(0)
    
    return threshold(dataframe, 5)
    

def threshold(dataframe, k):
    data = dataframe.sum()
    c = []
    for d in data:
        if d > k:
            c.append(True)
        else:
            c.append(False)
    dataframe = dataframe[dataframe.columns[c]]
    
    return dataframe

def selection_best_feat(features, k):
    s = SelectKBest(chi2, k=k)
    X_new = s.fit_transform(features, labels)
    return pd.DataFrame(X_new, columns=features.columns[s.get_support()])

# Fitness Functions

In [0]:
def get_columns(train, features):
    c = train.columns
    columns = []
    for i in range(len(features)):
        if features[i] == 1:
            columns.append(c[i - 2])

    return columns

def fit_acc_feat(chromossome, train_dt, train_lb, test_dt, test_lb, model):
    columns = get_columns(train_dt, chromossome)
    train_dt = train_dt[columns]
    test_dt = test_dt[columns]
    model.fit(train_dt, train_lb)
    predicts = model.predict(test_dt)
    acc = accuracy_score(test_lb, predicts)
    fitness = 0.95 * acc + 0.05 * (1 - (np.sum(chromossome) / len(chromossome)))
    return fitness
    
    

# Super Classe

In [0]:
class EvolutionaryAlgorithm():
    def __init__(self):
        self.__chromosomes = []
        self.__fitness = []
        self.__best = []

    def _chromossomes(self, chromosomes, fitness):
        self.__chromosomes.append([list(i) for i in chromosomes])
        self.__fitness_value.append([list(i) for i in fitness])

    def get_chromosomes(self):
        return self.__chromossomes
    
    def get_fitness_values(self):
        return self.__fitness
    
    def _set_best(self, best):
        self.__best = best
        
    def get_best(self):
        return self.__best

# Algoritmo Genético

In [0]:
class AG(EvolutionaryAlgorithm):
    def __init__(self, nPop, nGeneration, txMut, txCross, dimensions, fitness, model):
        super(AG, self).__init__()
        self.__genes = None
        self.nPop = nPop
        self.nGeneration = nGeneration
        self.dimensions = dimensions
        self.fitness = fitness
        self.txMut = txMut
        self.txCross = txCross
        self.model = model
        
    def run(self, train_dt, train_lb, valid_dt, valid_lb):
      
        self.__genes = np.random.randint(2, size=(self.nPop, self.dimensions))
        
        
        
        fit_value = np.array([self.fitness(x, train_dt, train_lb, valid_dt, valid_lb,
                                              self.model) for x in self.__genes])
        
        self._chromossomes(self.__genes, fit_value)
        for i in range(self.nGeneration):
            print("Generation ", str(i+1))
            newGeneration = self.tournament_selection(fit_value, self.__genes,
                                                      self.txCross)
            
            self.__genes = self.mutation(newGeneration, self.txMut)
            
            fit_value = np.array([self.fitness(x, train_dt, train_lb, valid_dt, valid_lb,
                                                  self.model) for x in self.__genes])
            
            self._chromossomes(self.__genes, fit_value)
        
    def elitism(pop, fit_value):
        f = fit_value.copy()
        idx1 = np.argmax(f)
        f[idx1] = 0
        idx2 = np.argmax(f)
        
        return pop[idx1], pop[idx2]
        
    def tournament_selection(self, fit_value, pop, txCross):
        newGeneration = []
        best1, best2 = elitism(pop, fit_value)
        newGeneration.append(best1)
        newGeneration.append(best2)
        for i in range(int(self.nPop/2)-1):
            idx = list(range(self.nPop))
            selected = rnd.choices(population=idx, k=2)
            fathers = [(fit_value[k], pop[k]) for k in selected]
            selected = sorted(fathers, key=lambda x: x[0], reverse=True)
            father1 = selected[0][1]
            
            selected = rnd.choices(population=idx, k=2)
            fathers = [(fit_value[k], pop[k]) for k in selected]
            selected = sorted(fathers, key=lambda x: x[0], reverse=True)
            father2 = selected[0][1]

            son1, son2 = self.crossover(father1, father2, txCross)
            
            newGeneration.append(son1)
            newGeneration.append(son2)
        return newGeneration
        
    def mutation(self, pop, txMut):
        for i in range(2, self.nPop):
            for j in range(self.dimensions):
                if rnd.random() <= txMut:
                    pop[i][j] = 1 * (pop[i][j] == 0)
        return pop
            
    def crossover(self, father1, father2, txCross):
        if rnd.random() <= txCross:
            mask = np.random.randint(2, size=self.dimensions)
            son1 = []
            son2 = []
            for j in range(self.dimensions):
                if mask[j] == 1:
                    son1.append(father1[j])
                    son2.append(father2[j])
                else:
                    son1.append(father1[j])
                    son2.append(father2[j])
        else:
            son1 = father1
            son2 = father2
        return son1, son2
        

# Estratégias Evolutivas

In [0]:
class EE(EvolutionaryAlgorithm):
    def __init__(self, nPop, nLambda, nGeneration, txMut, txCross, dimensions, fitness, model):
        super(EE, self).__init__()
        self.__genes = None
        self.nPop = nPop
        self.nLambda = nLambda
        self.nGeneration = nGeneration
        self.dimensions = dimensions
        self.fitness = fitness
        self.txMut = txMut
        self.txCross = txCross
        self.model = model
        
    def two_membersEE(self, train_dt, train_lb, valid_dt, valid_lb):
        success = 0
        nMut = 0
        x = np.random.rand(1, self.dimensions)
        stds = np.random.rand(1, self.dimensions)
        for i in range(self.nGeneration):
            xl = [(atrib + rnd.multivariate(0, s)) for atrib in x for s in stds]
            nMut += 1

            v = success/nMut

            if v > 0.2:
                stdl = std * 0.82
            elif v < 0.2:
                stdl = std / 0.82

            fit = fitness(x, train_dt, train_lb, valid_dt, valid_lb)
            fitl = fitness(xl, train_dt, train_lb, valid_dt, valid_lb)

            if fitl > fit:
                success += 1
                x = xl
                stds = stdl
                pass
            pass
        return x, stds
    
    def roulette_wheel_selection(fitness, k):
        total_fit = float(sum(fitness))
        relative_fitness = [f/total_fit for f in fitness]
        prob = [sum(relative_fitness[:i+1]) 
                for i in range(len(relative_fitness))]
        
        chosen = []
        for n in range(k):
            r = random.random()
            for i in range(self.nPop):
                if r <= prob[i]:
                    chosen.appen(i)
                    break
   
        return chosen
            
    def crossover(self, x, stds, fitness):
        newGeneration = []
        for i in range(int(self.nLambda/2)):
            pais = self.roulette_selection(fitness, 2)
            
            filho1 = []
            stdf1 = []
            filho2 = []
            stdf2 = []
            for i in range(self.dimensions):
                b = rnd.random()
                if b <= 0.5:
                    filho1.append(x[pais[0]][i])
                    filho2.append(x[pais[1]][i])
                    stdf1.append(stds[pais[0]][i])
                    stdf2.append(stds[pais[1]][i])
                else:
                    filho2.append(x[pais[0]][i])
                    filho1.append(x[pais[1]][i])
                    stdf2.append(stds[pais[0]][i])
                    stdf1.append(stds[pais[1]][i])
                    
            newGeneration.append((filho1, stdf1))
            if len(newGeneration) == self.nLambda:
                return newGeneration
            newGeneration.append((filho2, stdf2))
            
        return newGeneration
    
    def mutation(self, chromossome):
        n_chrom = []
        for i in range(self.dimensions):
            n_chrom.append(chrom[1][i]*np.exp(rnd.multivariate(0, 1)))
            n_chrom.append(chrom[0][i] + rnd.multivariate(0, chrom[1][i]))
        
        return n_chrom
        
    def mi_lambdaEE(self, train_dt, train_lb, valid_dt, valid_lb):
        x = np.random.rand(self.nPop, self.dimensions)
        stds = np.random.rand(self.nPop, self.dimensions)
        
        pop_denorm = [0 if x[i][j] < 0.5 else 1 for i in 
                      range(self.nPop) for j in range(self.dimensions)]
        fit = np.asarray([self.fitness(ind, train_dt, train_lb, valid_dt, valid_lb,
                                       self.model) for ind in pop_denorm])
        for k in range(self.nGeneration):
            newGeneration = self.crossover(x, stds, fit)

            for i in range(self.nLambda):
                newGeneration[i] = self.mutation(newGeneration[i])

            x = newGeneration[0]
            stds = newGeneration[1]

            pop_denorm = [0 if x[i][j] < 0.5 else 1 for i in 
                          range(self.nPop) for j in range(self.dimensions)]

            fit = np.asarray([self.fitness(ind, train_dt, train_lb, valid_dt, valid_lb,
                                           self.model) for ind in pop_denorm])
            
            x_v = zip(fit, (x, stds))
            
            s = sorted(x_v, key=lambda x: x[0], reverse=True)
            s = s[:self.nPop]
            x = s[1][0]
            stds = s[1][1]
            fit = s[0]
        
        
        

# Programação Evolutiva

In [0]:
# Inicializar população(x,n)
# Avalia população
# enquanto(critério de parada)
#     cada indivíduo pai gera um filho
#     avaliar filhos
#     fazer a união de pais e filhos
#     comparar cada elemento da união com q indivíduos: incrementa vitória
#     para quem ganha
#     selecionar os indivíduos que possuem mais vitórias
def init_pop(npop, dim):
    pop = np.random.rand(npop, dim)
    std = np.random.rand(npop, dim)
    return (pop, std)

def PE(fitness, npop, dim, niter, q):
    pop = init_pop(npop, dim)
    chrom = pop[0]
    std = [1]
    fit = [fitness(x) for x in chrom]
    for i in range(nIter):
        n1 = np.random.rand(npop, dim)
        xl = chrom + std * n1
# init.population <- function(pop.size,dimension,lb,ub){
#   pop <- matrix(runif(pop.size*dimension),nrow=pop.size)
#   pop <- lb + pop * (ub-lb)
#   std.dev <- matrix(runif(pop.size*dimension),nrow=pop.size)
#   return(list(pop = pop,std.dev = std.dev))
# }

    
# EP <- function(func,pop.size, dimension,lb,ub,num.it,q){
#   tau  <- sqrt(2 * sqrt(pop.size))^-1
#   taul <- sqrt(2 * pop.size)^-1
#   pop <- init.population(pop.size, dimension,lb,ub) 
#   x <- pop$pop
#   std.dev <- pop$std.dev
#   fit <- apply(x,1,func)
#   for(i in 1:num.it){
#     #Cria filhos usando Cauchy mutation
#     n1 <- matrix(rcauchy(pop.size*dimension),nrow=pop.size) 
#     xl <- x + std.dev * n1
#     #Verifica os limites de cada gene dos filhos 
#     idx <- which(xl < lb, arr.ind=TRUE)
#     xl[idx] <- lb[idx[,2]]
#     idx <- which(xl > ub, arr.ind=TRUE)
#     xl[idx] <- ub[idx[,2]]
#     #Cria novos desvios    
#     n1 <- rnorm(pop.size, mean = 0, sd = 1)
#     n.norm <- rnorm(1, mean = 0, sd = 1)
#     std.devl <- std.dev * exp(taul*n.norm + tau*n1)
#     fitl <- apply(xl,1,func)
#     #Faz a união entre pais e filhos
#     tmp.pop <- rbind(xl,x)
#     tmp.fit <- c(fitl,fit)
#     tmp.std <- rbind(std.devl,std.dev)
    
#     #Computa as vitórias de cada individuo
#     win <- rep(NA, pop.size*2)
#     for(j in 1:(2*pop.size)){ 
#       idx <- sample(1:(2*pop.size),q)
#       win[j] <- length(which(tmp.fit[j] < tmp.fit[idx]))
#     }
#     #Une as duas populações e ordena pelo numero de vitorias
#     #Em seguida trunca-se pelo tamanho da população
#     merge <- cbind(win,tmp.fit,tmp.pop,tmp.std)
#     merge <- merge[order(merge[,1],decreasing = TRUE),]
#     x <- merge[1:pop.size,3:(dimension+2)]
#     fit <- merge[1:pop.size,2]
#     std.dev <- merge[1:pop.size,(dimension+3):(dimension*2 + 2)]
#   }
#   return(list(pop=x,fit=fit,eta=std.dev))
# }
class PE():
    def __init__(self, nPop, nGeneration, txMut, txCross, dimensions, fitness, model):
        self.__pop = None
        self.nPop = nPop
        self.nGeneration = nGeneration
        self.dimensions = dimensions
        self.fitness = fitness
        self.txMut = txMut
        self.txCross = txCross
        self.model = model
        
    def run(self, train_dt, train_lb, valid_dt, valid_lb):
        x = np.random.rand(self.nPop, self.dimensions)
        stds = np.random.rand(self.nPop, self.dimensions)
        self.__pop = 

# Evolução Diferencial

In [0]:
class DE():
    def __init__(self, nPop, nIter, txMut, txCross, dimensions, fitness, model):
        self.__vectors = None
        self.nPop = nPop
        self.nIter = nIter
        self.dimensions = dimensions
        self.fitness = fitness
        self.txMut = txMut
        self.txCross = txCross
        self.model = model
        self.__chromosomes = []
        self.__best = []

    def _chromossomes(self, chromosomes):
        self.__chromosomes.append([list(i) for i in chromosomes])

    def get_chromosomes(self):
        return self.__chromossomes
    
    def _set_best(self, best):
        self.__best = best
        
    def get_best(self):
        return self.__best
        
    def run(self, train_dt, train_lb, valid_dt, valid_lb):
        self.__vectors = np.random.rand(self.nPop, self.dimensions)
        self._set_chromossomes(self.__vectors)
        pop_denorm = [0 if self.__vectors[i][j] < 0.5 else 1 for i in 
                      range(self.nPop) for j in range(self.dimensions)]
        fit = np.asarray([self.fitness(ind, train_dt, train_lb, valid_dt, valid_lb,
                                       self.model) for ind in pop_denorm])
        best_idx = np.argmax(fit)
        best = pop_denorm[best_idx]
        self._set_best = {best_idx: best}
        for i in range(self.nIter):
            for j in range(self.nPop):
                idxs = [idx for idx in range(self.nPop) if idx != j]
                a, b, c = self.__vectors[rnd.sample(idxs, 3)]
                mutant = np.clip(a + self.txMut * (b - c), 0, 1)
                cross_points = np.random.rand(self.dimensions) < self.txCross
                if not np.any(cross_points):
                    cross_points[np.random.randint(0, self.dimensions)] = True
                trial = np.where(cross_points, mutant, self.__vectors[j])
                trial_denorm = [0 if self.__vectors[i][j] < 0.5 else 1 for i in 
                               range(self.nPop) for j in range(self.dimensions)]
                f = self.fitness(trial_denorm, train_dt, train_lb, valid_dt, valid_lb,
                                 self.model)
                if f < fit[j]:
                    fit[j] = f
                    self.__vectors[j] = trial
                    if f < fit[best_idx]:
                        best_idx = j
                        best = trial_denorm
                        self._set_best({best_idx: best})
            self._set_chromossomes(self.__vectors)

# Experimentos

In [0]:
# GSO-SVM for Feature Selection & Kernel Parameter Optimization população 48, gerações 80
# Genetic algorithms for feature selection and weighting, a review and study população 75, gerações 25
# Feature Reduction Based on Genetic Algorithm and HybridModel for Opinion Mining população 50, gerações não fala
# nº de features sem threshold = 156652
# nº de features com df > 5 = 5000
# nº de features com df > 3 = 8622
# Estratégias Evolutivas Aplicadas à Resolução de Otimização Multimodal população 20, lambda 100, 1000 gerações

arq = open('books.pk', 'rb')
reviews = pickle.load(arq)
data, labels = reviews[0], reviews[1]

if not os.path.exists("features.txt"):
    get_all_features(data)
    dataframe = get_features_disk()
else:
    dataframe = get_features_disk()
dim = len(dataframe.columns)
print(dim)

for i in range(50):
    pass
    

# Testes

In [0]:
arq = open('books.pk', 'rb')
reviews = pickle.load(arq)
data, labels = reviews[0], reviews[1]

dataframe = get_features_disk()
dim = len(dataframe.columns)

train_dt, test_dt, train_lb, test_lb = train_test_split(dataframe, labels, train_size=0.8, test_size=0.2,
                                                  shuffle=True, stratify=labels)

train_dt, valid_dt, train_lb, valid_lb = train_test_split(train_dt, train_lb, train_size=0.8, test_size = 0.2,
                                                          shuffle=True, stratify=train_lb)
lr = LogisticRegression(solver='lbfgs', n_jobs=-1)
ag = AG(nPop=5, nGeneration=10, txMut=0.01, txCross=0.9, dimensions=dim, fitness=fit_acc_feat, model=lr)
ag.run(train_dt, train_lb, valid_dt, valid_lb)
c = ag.get_chromossomes().copy()
k = zip(c, ag.get_fitness())
new = sorted(k, key=lambda x: x[1])
print(new[0])

ed = ED(nPop=5, nIter=10, txMut=0.8, txCross=0.7, dimensions=dim, fitness=fit_acc_feat, model=lr)
ed.run(train_dt, train_lb, valid_dt, valid_lb)

In [0]:
from sklearn.feature_selection import chi2, SelectKBest
# import pickle
# arq = open("books.pk", "rb")
# reviews = pickle.load(arq)
# data, labels = reviews[0], reviews[1]

# df = get_all_features(data)
df = get_features_disk()
print(len(df.columns))
print(df)
# nV = 2000
# s = SelectKBest(chi2, k=20)
# X_new = s.fit_transform(df, labels)
# print(df.columns[s.get_support()])


